## Clean and Merge Data
This code merges the yearly CDC prescription data into one csv, combining the separate files for 2006 through 2016
into one and appending a column with the appropriate year. It also incorporates latitude and longitude data for indiana, which is also saved to a separate csv. 


In [2]:
import os
import pandas as pd 
import statistics

In [26]:
def find_avg_mortality(mort_range): #Calculating the average mortality rate given the Estimated Age-adjusted Death Rate 
    """
    Parameters:
    mort-range - a estimated range of average mortality rates in a county, given in the form of , '<2', #-#' or '30+'
    
    Returns:
    The mean of the mort-range, or 30 if the range is >30 
    """
    if '+' in mort_range:
        return 30.0
    elif '<' in mort_range:
        return float(statistics.mean([0,2]))
    else:
        separated = mort_range.split('-')
        lo, hi  = float(separated[0]), float(separated[1])
        return statistics.mean([lo, hi])
    
def clean_mortality_data(df):
    ''' Takes the  mortality data from CDC and cleans it (using find_avg_mortality) then filters it '''
    df['avg_mortality_rate'] = df['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'].apply(find_avg_mortality)
    #For my research we only need the years 2006 to 2016 
    clean_df = df[df['Year'] >=2005]
    return clean_df 

def merge_datasets(clean_prescription_df, clean_mortality_df): 
    """
    Combines our prescription and morality datasets and modifies the header names to make them more usable.
    
    """
    ## Creating Merged Data Frame 
    df = pd.merge(clean_prescription_df, clean_mortality_df, left_on = ['FIPS County Code', 'Year'], right_on = ['FIPS', 'Year']) #Merging based on County Code and Year 
    df = df.drop(['FIPS County Code', 'County_y'], axis=1) #Dropping Duplicates

    #Renaming 
    df.columns = ['county', 'state_abbrv', 'prescription_rate', 'year', 'fips', 'state', 'fips_state', 'population', 'age_adjusted_mortality_range', 'avg_mortality_rate']

    #Reordering the columns
    df = df[['county', 'fips', 'state_abbrv', 'state', 'fips_state', 'year', 'population', 'prescription_rate', 'age_adjusted_mortality_range', 'avg_mortality_rate']]

    return df 

def yearly_change(source_df, source_col, change_col, sort_col, year_col):
    """ Calculates the change from the prior year for the given column
    Parameters:
    df - the pd.DataFrame you want to modify
    source_col - the column you want to calculate annual change for
    change_col - the column to hold these calculations
    
    Return:
    df - the dataframe with the updated columns 
    """
    #Sorting columns first so later iterating will work
    df = source_df.sort_values([sort_col,year_col],ascending=[True,True])

    #Getting the index for source column for use in iloc later
    source_col_index = df.columns.get_loc(source_col) 
    fips_index = df.columns.get_loc(sort_col)
    year_index = df.columns.get_loc(year_col)
    df[change_col] = None
    change_col_index = df.columns.get_loc(change_col)
    
    for row in range(len(df)):
        if df.iloc[row,fips_index] == df.iloc[row-1, fips_index] and df.iloc[row - 1, year_index] < df.iloc[row, year_index] : #checking that it is the same county code 
            df.iloc[row, change_col_index] = df.iloc[row, source_col_index] - df.iloc[row - 1, source_col_index]
    return df 

def fix_fips(df, fips_col ='fips', fips_size=5):
    """
    This is a function to fix the FIPS column. The fips_size refers to whether you are trying to 
    fix a county or state level fips; pass the size of the fips code you are trying to get. 
    
    """
    df[fips_col] = df[[fips_col]].applymap(lambda x: ('000' + str(x))[-fips_size:])
    return df

In [3]:
# Reading in Prescription and Mortality Files
# Assumes you are in 'code' and 'raw_data' is another folder in parent directory

prescription_fname = "../raw_data/cdc_opioid_prescribing_rate.csv" 
with open (prescription_fname) as f:
    prescription_df = pd.read_csv(f, na_values="–")


mortality_fname =  "../raw_data/NCHS_-_Drug_Poisoning_Mortality_by_County__United_States.csv"
with open (mortality_fname) as f:
    mortality_df = pd.read_csv(f)


In [4]:
#Preliminary_cleaning the Prescription Data -- simple  
prescription_df = prescription_df.dropna()
clean_prescription_df = prescription_df[prescription_df['Year'] >= 2005]
clean_prescription_df.head()

,County,State,FIPS County Code,Prescribing Rate,Year
2,"Anchorage, AK",AK,2020,71.5,2006
7,"Fairbanks North Star, AK",AK,2090,54.7,2006
10,"Juneau, AK",AK,2110,95.3,2006
11,"Kenai Peninsula, AK",AK,2122,89.1,2006
12,"Ketchikan Gateway, AK",AK,2130,144.4,2006


In [6]:
clean_mortality_df = clean_mortality_data(mortality_df)
clean_mortality_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37632 entries, 6 to 56447
Data columns (total 8 columns):
FIPS                                                            37632 non-null int64
Year                                                            37632 non-null int64
State                                                           37632 non-null object
FIPS State                                                      37632 non-null int64
County                                                          37632 non-null object
Population                                                      37632 non-null object
Estimated Age-adjusted Death Rate, 16 Categories (in ranges)    37632 non-null object
avg_mortality_rate                                              37632 non-null float64
dtypes: float64(1), int64(3), object(4)
memory usage: 2.6+ MB


In [8]:
df = merge_datasets(clean_prescription_df, clean_mortality_df)
df.head()

,county,fips,state_abbrv,state,fips_state,year,population,prescription_rate,age_adjusted_mortality_range,avg_mortality_rate
0,"Anchorage, AK",2020,AK,Alaska,2,2006,"280,085",71.5,12-13.9,12.95
1,"Fairbanks North Star, AK",2090,AK,Alaska,2,2006,"90,545",54.7,8-9.9,8.95
2,"Juneau, AK",2110,AK,Alaska,2,2006,"30,808",95.3,8-9.9,8.95
3,"Kenai Peninsula, AK",2122,AK,Alaska,2,2006,"52,253",89.1,12-13.9,12.95
4,"Ketchikan Gateway, AK",2130,AK,Alaska,2,2006,"13,492",144.4,8-9.9,8.95


In [12]:
#Clculating the changes per year  using our year_change function
df = yearly_change(df, 'avg_mortality_rate', 'change_mortality_rate', 'fips', 'year') 
df = yearly_change(df, 'prescription_rate', 'change_prescription_rate', 'fips', 'year')

## Fix the Population Data -- currently a string with a comma in it, need it to be an actual number
df['population'] = df['population'].map(lambda x: int(x.replace(',', '')))
df['population'][0]

#Using our fix_fips() function to make sure that our fips codes are in the right format
df = fix_fips(df)
df = fix_fips(df, 'fips_state', 2)

280085

In [14]:
#Checking our final data frame to see if it looks right: 
print(df.head(5))
print(df.info())

            county   fips state_abbrv    state fips_state  year  population  \
8      Autauga, AL  01001          AL  Alabama         01  2006       51328   
2757   Autauga, AL  01001          AL  Alabama         01  2007       52405   
5497   Autauga, AL  01001          AL  Alabama         01  2008       53277   
8249   Autauga, AL  01001          AL  Alabama         01  2009       54135   
10993  Autauga, AL  01001          AL  Alabama         01  2010       54660   

       prescription_rate age_adjusted_mortality_range  avg_mortality_rate  \
8                  134.8                        6-7.9                6.95   
2757               135.8                        6-7.9                6.95   
5497               144.9                        6-7.9                6.95   
8249               147.5                        6-7.9                6.95   
10993              151.7                        8-9.9                8.95   

      change_mortality_rate change_prescription_rate  
8      

In [15]:
#Finally, for this project we will just be looking at indiana 
indiana_df = df[df['state']=='Indiana']

#### Process Indiana FIPS to Latitude and Longitude

In [16]:
"""
Reading in our data. This solution was suggested 
https://stackoverflow.com/questions/42868735/how-do-i-convert-from-census-fips-to-lat-lon
and utilizes data from 
https://geonames.usgs.gov/domestic/download_data.htm
"""
geo_df = pd.read_csv('../raw_data/IN_FedCodes_20190301.txt', delimiter='|')
#Selecting just our data and renaming
geo_df = geo_df[['STATE_ALPHA', 'STATE_NUMERIC', 'COUNTY_NUMERIC', 'COUNTY_NAME', 'PRIMARY_LATITUDE', 'PRIMARY_LONGITUDE']]
geo_df.columns = ['state_abbrv', 'fips_state', 'fips', 'county_short', 'latitude', 'longitude']

In [17]:
geo_df = fix_fips(geo_df, 'fips', 3)
geo_df = fix_fips(geo_df, 'fips_state', 2)
geo_df['fips'] = geo_df['fips_state'].str.cat(geo_df['fips']) # Using the series concatenator 
geo_df.head()

,state_abbrv,fips_state,fips,county_short,latitude,longitude
0,IN,18,18115,Ohio,38.905338,-84.987451
1,IN,18,18177,Wayne,39.733103,-84.963017
2,IN,18,18003,Allen,41.000881,-85.318029
3,IN,18,18071,Jackson,38.978108,-86.058871
4,IN,18,18097,Marion,39.655601,-85.966927


In [19]:
#We have so many different individual ones within each county that we need to combine them into 1. This 
#resulting longitude and latitude is the approximte center of each county 
geo_df = geo_df.groupby('fips').mean()
geo_df.head()

,latitude,longitude
fips,,
17045,39.545313,-87.530578
17091,41.189757,-87.526146
18001,40.746298,-84.956258
18003,41.085665,-85.076343
18005,38.621598,-84.590417


In [20]:
#Merging the two datasets and replacing indiana_df with the full result
indiana_df = indiana_df.merge(geo_df, on='fips') # did tests to make sure that everything transfers correctly: it does
print(indiana_df.head())
print(indiana_df.info())


,county,fips,state_abbrv,state,fips_state,year,population,prescription_rate,age_adjusted_mortality_range,avg_mortality_rate,change_mortality_rate,change_prescription_rate,latitude,longitude
0,"Adams, IN",18001,IN,Indiana,18,2006,33887,64.1,4-5.9,4.95,None,None,40.746298,-84.956258
1,"Adams, IN",18001,IN,Indiana,18,2007,33962,66.3,6-7.9,6.95,2,2.2,40.746298,-84.956258
2,"Adams, IN",18001,IN,Indiana,18,2008,34214,68.4,6-7.9,6.95,0,2.1,40.746298,-84.956258
3,"Adams, IN",18001,IN,Indiana,18,2009,34351,68.6,6-7.9,6.95,0,0.2,40.746298,-84.956258
4,"Adams, IN",18001,IN,Indiana,18,2010,34455,65.7,8-9.9,8.95,2,-2.9,40.746298,-84.956258


In [25]:
#writing to CSV files so we can then use the clean data later without always having to reprocess 
df.to_csv('../clean_data/opioid_data_2006-2016.csv')
indiana_df.to_csv('../clean_data/opioid_data_indiana_2006-2016.csv')